In [1]:
from pyspark.sql import SparkSession, functions as F

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/24 19:47:13 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 172.25.24.22 instead (on interface eth0)
22/08/24 19:47:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/24 19:47:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/24 19:47:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql.functions import *


In [3]:
yellow_spark = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_credit')

In [4]:
# create dataframes to aggregate data based in if it is a weekend or not and also aggregate based on win or lost

yellow_weekday = yellow_spark.filter( F.col("is_weekend") == True )
yellow_weekend = yellow_spark.filter( F.col("is_weekend") == False)

yellow_win = yellow_spark.filter( F.col("win") == "Yes" )
yellow_lose = yellow_spark.filter( F.col("win") == "No" )



In [6]:
# counts for wins 

yellow_win.count()

1330808

In [7]:
# counts for lost 
# higher number is expected as the knicks lost a more substantial amount of games this season compared
# to wins

yellow_lose.count()

5085064

In [5]:
# average tip amount per hour in a weekday
aggregate_weekday = (yellow_weekday
     .groupBy(hour("tpep_pickup_datetime").alias("hour"))
     .agg(round(mean("tip_amount"),4).alias("mean_tip"))
     .orderBy("hour")
    )
aggregate_weekday.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_weekday')

In [6]:
# average tip amount per hour in a weekend

aggregate_weekend = (yellow_weekend
     .groupBy(hour("tpep_pickup_datetime").alias("hour"))
     .agg(round(mean("tip_amount"),4).alias("mean_tip"))
     .orderBy("hour")
    )

aggregate_weekend.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_weekend')

In [7]:
# average tip amount per hour when the knicks win

aggregate_win = (yellow_win
     .groupBy(hour("tpep_pickup_datetime").alias("hour"))
     .agg(round(mean("tip_amount"),4).alias("mean_tip"))
     .orderBy("hour")
    )

aggregate_win.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_win')

In [8]:
# average tip amount per hour when the knicks lose

aggregate_lose = (yellow_lose
     .groupBy(hour("tpep_pickup_datetime").alias("hour"))
     .agg(round(mean("tip_amount"),4).alias("mean_tip"))
     .orderBy("hour")
    )

aggregate_lose.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_lose')

In [9]:
# average tip amount based on when the game starts

aggregate_start = (yellow_spark
                       .groupBy(F.col("Start(ET)").alias("start_time"))
                       .agg(round(mean("tip_amount"),4).alias("mean_tip"))
                       .orderBy("start_time"))
aggregate_start.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_start')

In [10]:
# aggregate by Pickup_location

aggregate_PU = (yellow_spark
                       .groupBy(F.col("PULocationID").alias("Pickup_location"))
                       .agg(round(mean("tip_amount"),4).alias("mean_tip"))
                       .orderBy("Pickup_location"))
aggregate_PU.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_PU')

In [11]:
# Aggregate by dropoff location

aggregate_DO = (yellow_spark
                       .groupBy(F.col("DOLocationID").alias("Dropoff_location"))
                       .agg(round(mean("tip_amount"),4).alias("mean_tip"))
                       .orderBy("Dropoff_location"))
aggregate_DO.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_DO')

In [12]:
# take 10% sample from main data

SAMPLE_SIZE = 0.10

yellow_sample = yellow_spark.sample(SAMPLE_SIZE, seed=0)

yellow_sample.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_sample')
